# Back to refactoring titan algorithms

- Work on loading and syncing running algos
- Work on signals and distributed human intervention
- Work on buy and start for reacting to signals
- Work on exchange based harvester signals
- Work on backtesting
- Normalize candles that are realtime based, i.e. round to the next nearest fraction of an hour
- Storing results with more data points like catalyst pickles

# Found

- dual moving xmr-btc bittex 5m 3-15

# Bugs

- 1m candles throw list index error
- capital_base gets ignored if less than purchase cost

In [15]:
from os import getcwd
from kryptobot.portfolio.manager import Manager

config = getcwd() + '/config.json'
manager = Manager(config=config)

params = {
#     'strategy': 'dema_crossover',
    'strategy': 'dual_simple_moving_average',
    'type': 't2',
    'default': {
        'interval': '5m',
        'exchange': 'bitfinex',
        'base_currency': 'NEO',
        'quote_currency': 'BTC',
        'is_simulated': True,
#         'start': '2017-06-15',
#         'end': '2018-06-15',
    },
    'limits': {
        'capital_base': 1000,
        'order_quantity': 100,
        'position_limit': 1000,
        'profit_target_percentage': 1.2,
        'fixed_stoploss_percentage': .95,
        'trailing_stoploss_percentage': .90
    },
    'custom': {
        'short_window': 3,
        'long_window': 36,
#         'threshold': 0.02
    },
    'portfolio': {
        'name': 'default'
    }
}

manager.run_strategy(params)

In [16]:
from os import getcwd
from kryptobot.portfolio.manager import Manager

config = getcwd() + '/config.json'
manager = Manager(config=config)

results = manager.get_results('024aa804-2f47-4086-8076-edeb0c336c97')
# results.head(300)
results.tail(300)

,base_balance,close,fma,high,low,open,positions,quote_balance,sell_all_balance,sma,timestamp,volume,volume_change
0,0,0.005699,0.005883,0.005699,0.005699,0.005699,0,1000.00000,1000.000000,0.006335,2017-09-07 10:10:00,120.000000,-70.00
1,0,0.005876,0.005725,0.005896,0.005700,0.005700,0,1000.00000,1000.000000,0.006312,2017-09-07 10:15:00,438.221238,265.18
2,0,0.006000,0.005858,0.006000,0.005899,0.005899,0,1000.00000,1000.000000,0.006291,2017-09-07 10:20:00,1379.925000,214.89
3,0,0.006292,0.006056,0.006292,0.006030,0.006100,0,1000.00000,1000.000000,0.006278,2017-09-07 10:25:00,487.584534,-64.67
4,0,0.006400,0.006231,0.006499,0.006295,0.006300,0,1000.00000,1000.000000,0.006272,2017-09-07 10:30:00,1117.797890,129.25
5,0,0.006382,0.006358,0.006500,0.005819,0.006380,0,1000.00000,1000.000000,0.006264,2017-09-07 10:35:00,3413.791492,205.40
6,0,0.006737,0.006506,0.006798,0.005986,0.006475,0,1000.00000,1000.000000,0.006264,2017-09-07 10:40:00,1773.351433,-48.05
7,100,0.006499,0.006539,0.006718,0.006499,0.006718,1,999.40140,1000.062250,0.006261,2017-09-07 10:45:00,1161.644637,-34.49
8,100,0.006502,0.006579,0.006509,0.006182,0.006493,1,999.40140,1000.035950,0.006264,2017-09-07 10:50:00,915.797352,-21.16
9,100,0.006500,0.006500,0.006520,0.006399,0.006484,1,999.40140,1000.047350,0.006269,2017-09-07 10:55:00,138.366551,-84.89
